# Test my own cityscpes code

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
from datetime import datetime
import os.path
import sys
import time

import numpy as np
from six.moves import xrange
import tensorflow as tf
import threading

from config import *
from dataset import pascal_voc, kitti, cityscapes
from utils.util import sparse_to_dense, bgr_to_rgb, bbox_transform
from nets import *

In [2]:
mc = cityscapes_squeezeDet_config()
mc.IS_TRAINING = True
mc.PRETRAINED_MODEL_PATH = '../squeezeDet-master/data/SqueezeNet/squeezenet_v1.1.pkl'#FLAGS.pretrained_model_path
model = SqueezeDet(mc)
print(mc)

dataset: cityscapes
dataset.upper: CITYSCAPES
Cannot find fire10/squeeze1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire10/expand1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire10/expand3x3 in the pretrained model. Use randomly initialized parameters
Cannot find fire11/squeeze1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire11/expand1x1 in the pretrained model. Use randomly initialized parameters
Cannot find fire11/expand3x3 in the pretrained model. Use randomly initialized parameters
Cannot find conv12 in the pretrained model. Use randomly initialized parameters
INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
{'NMS_THRESH': 0.4, 'LOSS_COEF_CONF_NEG': 100.0, 'KEEP_PROB': 0.5, 'IMAGE_WIDTH': 320, 'NUM_THREAD': 4, 'LOSS_COEF_CONF_POS': 75.0, 'GRID_POOL_HEIGHT': 7, 'PLOT_PROB_THRESH': 0.4, 'TOP_N_DETECTION': 64, 'LOSS_COEF_CLASS': 1.0, 'BGR_MEANS': array([[[103.939, 

In [3]:
imdb = cityscapes('train', '/home/zhengyuanbo/project/SQDT_ROOT/data/cityscapes/', mc)

In [4]:
mc.BATCH_SIZE = 5 # Originally 20, for the test I was modified to 5
image_per_batch, label_per_batch, box_delta_per_batch, aidx_per_batch, \
          bbox_per_batch = imdb.read_batch()
'''
print("image_per_batch:", image_per_batch,
      "label_per_batch:", label_per_batch,
      "box_delta_per_batch:", box_delta_per_batch,
      "aidx_per_batch:", aidx_per_batch,
      "bbox_per_batch:", bbox_per_batch)
'''
print ("aa")

read_batch
name: /home/zhengyuanbo/project/SQDT_ROOT/data/cityscapes/leftImg8bit/train/aachen/aachen_000004_000019_leftImg8bit.png
orig_h: 1024.0 orig_w: 2048.0
argsort: [ 809  801  794 ... 1211 1212    0]
aidx_set: set([])
aidx: 809
ov_idx: 809
argsort: [ 756  764  760 ... 1211 1212    0]
aidx_set: set([809])
aidx: 756
ov_idx: 756
argsort: [ 764  756  767 ... 1211 1212    0]
aidx_set: set([809, 756])
aidx: 764
ov_idx: 764
argsort: [ 764  756  758 ... 1213 1214    0]
aidx_set: set([809, 756, 764])
aidx_set: set([809, 756, 764])
aidx_set: set([809, 756, 764])
aidx: 758
ov_idx: 758
argsort: [ 747  755  751 ... 1210 1211    0]
aidx_set: set([809, 756, 764, 758])
aidx: 747
ov_idx: 747
argsort: [ 746  740  749 ... 1210 1211    0]
aidx_set: set([809, 747, 756, 764, 758])
aidx: 746
ov_idx: 746
argsort: [ 738  746  742 ... 1210 1211    0]
aidx_set: set([809, 746, 747, 756, 758, 764])
aidx: 738
ov_idx: 738
argsort: [ 765  773  769 ... 1211 1212    0]
aidx_set: set([738, 809, 746, 747, 756, 758,

orig_h: 1024.0 orig_w: 2048.0
argsort: [ 751  755  747 ... 1218 1219    0]
aidx_set: set([])
aidx: 751
ov_idx: 751
argsort: [ 742  746  745 ... 1218 1219    0]
aidx_set: set([751])
aidx: 742
ov_idx: 742
argsort: [ 742  738  746 ... 1217 1218    0]
aidx_set: set([742, 751])
aidx_set: set([742, 751])
aidx: 738
ov_idx: 738
argsort: [ 792  796  800 ... 1214 1215    0]
aidx_set: set([738, 742, 751])
aidx: 792
ov_idx: 792
argsort: [ 792  796  800 ... 1217 1218    0]
aidx_set: set([792, 738, 742, 751])
aidx_set: set([792, 738, 742, 751])
aidx: 796
ov_idx: 796
argsort: [ 800  792  794 ... 1210 1211    0]
aidx_set: set([792, 738, 796, 742, 751])
aidx: 800
ov_idx: 800
argsort: [ 810  818  814 ... 1212 1213    0]
aidx_set: set([800, 738, 742, 751, 792, 796])
aidx: 810
ov_idx: 810
argsort: [ 785  787  786 ... 1223 1224    0]
aidx_set: set([800, 738, 742, 810, 751, 792, 796])
aidx: 785
ov_idx: 785
argsort: [ 801  809  805 ... 1214 1215    0]
aidx_set: set([800, 738, 742, 810, 751, 785, 792, 796])
a

In [9]:
#print("image_per_batch:", image_per_batch[0])
imgb = image_per_batch[0] + mc.BGR_MEANS
imgb = imgb.astype(np.uint8) 
print (len(bbox_per_batch[0]))
print (bbox_per_batch[0])

cv2.namedWindow("imgb",0);
cv2.moveWindow("imgb",100,100)
cv2.resizeWindow("imgb", 640, 480)
cv2.imshow("imgb", imgb)

for i in range (len(bbox_per_batch[0])):
    Xmin = int(bbox_per_batch[0][i][0] - (bbox_per_batch[0][i][2]/2))
    Ymin = int(bbox_per_batch[0][i][1] - (bbox_per_batch[0][i][3]/2)) 
    Xmax = int(bbox_per_batch[0][i][0] + (bbox_per_batch[0][i][2]/2)) 
    Ymax = int(bbox_per_batch[0][i][1] + (bbox_per_batch[0][i][3]/2))
    cv2.rectangle(imgb, (Xmin, Ymin), (Xmax, Ymax), (0,255,255), 1)

cv2.namedWindow("img",0);
cv2.moveWindow("img",800,100)
cv2.resizeWindow("img", 640, 480)
cv2.imshow("img", imgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

24
[[185.88921283  69.47569113   4.8202138    3.81315539]
 [190.47619048  68.78932316   8.70748299   7.01620591]
 [194.2079689   69.1706387    6.53061224   7.47378456]
 [199.80563654  69.0943756    9.01846453   8.54146806]
 [201.20505345  71.00095329   6.21963071   7.16873213]
 [207.89115646  69.62821735  13.68318756   8.69399428]
 [216.98736638  73.97521449   4.8202138    5.79599619]
 [212.24489796  68.71306006   8.39650146   2.89799809]
 [209.52380952  69.01811249   7.93002915   4.11820782]
 [210.30126336  71.76358437  14.14965986   8.08388942]
 [232.22546161  66.04385129  36.85131195  21.65872259]
 [234.24684159  73.0600572   12.90573372  11.8970448 ]
 [249.32944606  71.76358437  29.69873664  14.7950429 ]
 [268.06608358  73.44137274  34.82993197  17.84556721]
 [282.75996113  75.88179218  35.60738581  16.93040991]
 [176.55976676  69.93326978   3.26530612   5.33841754]
 [168.86297376  70.61963775   5.2866861    4.57578646]
 [165.83090379  69.93326978   4.8202138    4.42326025]
 [172.9

In [6]:
print(mc.ANCHOR_BOX[0:19])
print(len(mc.ANCHOR_BOX))
img_tmp = cv2.imread('../data/cityscapes/leftImg8bit/train/aachen/aachen_000001_000019_leftImg8bit.png')
img_tmp = cv2.resize(img_tmp,(320, 160), interpolation=cv2.INTER_AREA)
for i in range (len(mc.ANCHOR_BOX[0:9])):
    Xmin = int(100 - (mc.ANCHOR_BOX[i][2]/2))
    Ymin = int(100 - (mc.ANCHOR_BOX[i][3]/2)) 
    Xmax = int(100 + (mc.ANCHOR_BOX[i][2]/2)) 
    Ymax = int(100 + (mc.ANCHOR_BOX[i][3]/2))
    cv2.rectangle(img_tmp, (Xmin, Ymin), (Xmax, Ymax), (0,255,255), 1)

cv2.imshow("img", img_tmp)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[15.23809524 14.54545455 10.         10.        ]
 [15.23809524 14.54545455 94.         45.        ]
 [15.23809524 14.54545455 29.         14.        ]
 [15.23809524 14.54545455 42.         22.        ]
 [15.23809524 14.54545455 10.         23.        ]
 [15.23809524 14.54545455 66.         44.        ]
 [15.23809524 14.54545455 57.         28.        ]
 [15.23809524 14.54545455 20.         43.        ]
 [15.23809524 14.54545455 18.         11.        ]
 [30.47619048 14.54545455 10.         10.        ]
 [30.47619048 14.54545455 94.         45.        ]
 [30.47619048 14.54545455 29.         14.        ]
 [30.47619048 14.54545455 42.         22.        ]
 [30.47619048 14.54545455 10.         23.        ]
 [30.47619048 14.54545455 66.         44.        ]
 [30.47619048 14.54545455 57.         28.        ]
 [30.47619048 14.54545455 20.         43.        ]
 [30.47619048 14.54545455 18.         11.        ]
 [45.71428571 14.54545455 10.         10.        ]]
1800


In [14]:
label_indices, bbox_indices, box_delta_values, mask_indices, box_values, \
          = [], [], [], [], []
aidx_set = set()
num_discarded_labels = 0
num_labels = 0
print ('label_per_batch:', label_per_batch)
print ('label_per_batch shape:', len(label_per_batch))
for i in range(len(label_per_batch)): # batch_size
    for j in range(len(label_per_batch[i])): # number of annotations
        num_labels += 1
        print('aidx_per_batch:', aidx_per_batch[i][j], 'aidx_set:', aidx_set)
        if (i, aidx_per_batch[i][j]) not in aidx_set:
            aidx_set.add((i, aidx_per_batch[i][j]))
            label_indices.append(
                    [i, aidx_per_batch[i][j], label_per_batch[i][j]])
            mask_indices.append([i, aidx_per_batch[i][j]])
            bbox_indices.extend(
                    [[i, aidx_per_batch[i][j], k] for k in range(4)])
            box_delta_values.extend(box_delta_per_batch[i][j])
            box_values.extend(bbox_per_batch[i][j])
        else:
            num_discarded_labels += 1

label_per_batch: [[0, 0, 0, 0, 2, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 2], [0, 0]]
label_per_batch shape: 5
aidx_per_batch: 6642 aidx_set: set([])
aidx_per_batch: 6732 aidx_set: set([(0, 6642)])
aidx_per_batch: 5976 aidx_set: set([(0, 6642), (0, 6732)])
aidx_per_batch: 6723 aidx_set: set([(0, 5976), (0, 6642), (0, 6732)])
aidx_per_batch: 6597 aidx_set: set([(0, 6723), (0, 5976), (0, 6642), (0, 6732)])
aidx_per_batch: 7398 aidx_set: set([(0, 6723), (0, 5976), (0, 6597), (0, 6642), (0, 6732)])
aidx_per_batch: 7479 aidx_set: set([(0, 5976), (0, 6642), (0, 6732), (0, 7398), (0, 6597), (0, 6723)])
aidx_per_batch: 9009 aidx_set: set([(0, 5976), (0, 6642), (0, 6732), (0, 7398), (0, 6597), (0, 6723), (0, 7479)])
aidx_per_batch: 9081 aidx_set: set([(0, 5976), (0, 6642), (0, 6732), (0, 9009), (0, 7398), (0, 6597), (0, 6723), (0, 7479)])
aidx_per_batch: 8100 aidx_s

In [5]:
l = [162.,  87.,   38.,  90., 258., 173.,\
            224., 108.,  78., 170.,   72.,  43.]
for i in range (len(l)):
    l[i] = l[i] * 0.25641
print (l)

[41.53842, 22.30767, 9.743580000000001, 23.076900000000002, 66.15378000000001, 44.35893000000001, 57.435840000000006, 27.692280000000004, 19.99998, 43.58970000000001, 18.46152, 11.025630000000001]


In [12]:
boxes = mc.ANCHOR_BOX
box = bbox_per_batch[0][0]

print('boxes', boxes)
print('box', box)
print('minimum()', np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]))
print('maximum()', np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]))


boxes [[ 15.23809524  14.54545455  10.          10.        ]
 [ 15.23809524  14.54545455  94.          45.        ]
 [ 15.23809524  14.54545455  29.          14.        ]
 ...
 [304.76190476 145.45454545  57.          28.        ]
 [304.76190476 145.45454545  20.          43.        ]
 [304.76190476 145.45454545  18.          11.        ]]
box [185.88921283  69.47569113   4.8202138    3.81315539]
minimum() [ 20.23809524  62.23809524  29.73809524 ... 188.29931973 188.29931973
 188.29931973]
maximum() [183.47910593 183.47910593 183.47910593 ... 276.26190476 294.76190476
 295.76190476]


In [11]:
print(mc.ANCHOR_BOX[1800-1])

[304.76190476 145.45454545  18.          11.        ]


In [17]:
imgt = cv2.imread('../data/cityscapes/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png')
imgt = cv2.resize(imgt,(320, 160), interpolation=cv2.INTER_AREA)
cv2.namedWindow("img",0);
cv2.moveWindow("img",800,100)
cv2.resizeWindow("img", 640, 480)
cv2.imshow("img", imgt)
cv2.waitKey(0)
cv2.destroyAllWindows()